In [28]:
import torch
import tqdm
import numpy as np
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
learning_rate = 3e-4
max_iter = 10000
eval_iters = 250
dropout  = 0.2

cuda


In [8]:
# 读取绿野仙踪文本
with open(r'..\llm_gpt\wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
vocabulary_size = len(chars)

In [9]:
# 分词
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)

In [10]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([75, 27, 63, 66, 63, 68, 56, 73,  1, 49, 62, 52,  1, 68, 56, 53,  1, 46,
        57, 74, 49, 66, 52,  1, 57, 62,  1, 38, 74,  0,  0,  0,  1,  1, 24,  1,
        29, 49, 57, 68, 56, 54, 69, 60,  1, 41, 53, 51, 63, 66, 52,  1, 63, 54,
         1, 43, 56, 53, 57, 66,  1, 24, 61, 49, 74, 57, 62, 55,  1, 24, 52, 70,
        53, 62, 68, 69, 66, 53, 67,  0,  1,  1,  1,  1, 57, 62,  1, 49, 62,  1,
        44, 62, 52, 53, 66, 55, 66, 63, 69, 62])


In [12]:
split_ratio = 0.8
n = int(split_ratio * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print(x)
print(y)

tensor([[71, 49, 67,  0, 62, 63,  1, 53],
        [63, 66,  1, 68, 56, 53, 73,  1],
        [53,  1, 26, 49, 50,  9, 31, 63],
        [66, 67,  1, 63, 66,  1, 70, 53]], device='cuda:0')
tensor([[49, 67,  0, 62, 63,  1, 53, 72],
        [66,  1, 68, 56, 53, 73,  1, 56],
        [ 1, 26, 49, 50,  9, 31, 63, 66],
        [67,  1, 63, 66,  1, 70, 53, 55]], device='cuda:0')


建立模型

In [13]:
import torch.nn as nn
from torch.nn import functional as F

In [30]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = loss.mean()
    model.train()
    return out

In [16]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, vocabulary_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocabulary_size)
model = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generate_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generate_chars)


VGin5!PZv:lZvcv5':i1)B2)1O.rDR8Z4:6LeOS(c2l﻿Mv5B-Pc-9Qvms0&Cf,N6(ksjEOS0WFjrS'k':Ba6sy,HKv G1R&17R'&"﻿Nus;uYlt8PVZCoI,YV P:VCW&IV6Q:)hUrKz0v"
!f"K9B8)(NMe8i54qopLbgkR07aznns(!;t4K4YAGo5ZUsUhQjnMky1nSxv56TC.UWAO9?3Z3;:xyfm1tIricZ-oagLVDNswhYBUUkkAV)JTontT!MeG1AW!!-?x84bnYNo9w3,pR)w:5QyJuK:.oinFIv6kS-﻿;NGg!Qjc1d2AjQrB:V-NP!PO k2'Q;"!P:.V"UZTYVe:S7GMk42Qo;;-M&qPG?Oylh6:Vx1r8?g"yjEQAl"!DLy),N-VqVfGsqN﻿cq.;kinteTem4UWJm"!vYl﻿U?9 iTCS -JtEMCTyz 9dRy,'p&p69SdR8d&-Jo﻿HDuYVHW&q2galr:5fl﻿Pn"""o!hNCWgUZ-Jn


创建优化器

In [31]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in tqdm.tqdm(range(max_iter)):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, loss {losses}')

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(loss.item())

  1%|▏         | 141/10000 [00:00<00:20, 491.04it/s]

step: 0, loss {'train': tensor(2.5138, device='cuda:0'), 'val': tensor(2.7890, device='cuda:0')}


  4%|▎         | 360/10000 [00:00<00:17, 565.81it/s]

step: 250, loss {'train': tensor(2.4724, device='cuda:0'), 'val': tensor(2.8174, device='cuda:0')}


  7%|▋         | 731/10000 [00:01<00:12, 732.65it/s]

...

step: 9500, loss {'train': tensor(2.4504, device='cuda:0'), 'val': tensor(2.8872, device='cuda:0')}


 99%|█████████▊| 9872/10000 [00:16<00:00, 645.87it/s]

step: 9750, loss {'train': tensor(2.3841, device='cuda:0'), 'val': tensor(3.0176, device='cuda:0')}


100%|██████████| 10000/10000 [00:16<00:00, 607.52it/s]

2.9814469814300537


In [25]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generate_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generate_chars)


nthth?8forwne B4K1e lidZu th us 6T,Yarimmpsag." S7:im.
"!Ne.G'che w erid.
hilo JP:"Nt?xYV,iY84YBr ou pusk ingll1YV6ke w tt V"!!T!wad 'F7hofeve fy DJwhoulof the0:zld!GPrs t b)4bus ndd,NM﻿)&DJis m thonglicebu7B:7M﻿RAxO.Kb d do  crdin me s'Zyorfu res
Can no j﻿dil .
"Nf tttrzZy? s)xpwed."
"im'::9us, d anqUonglwePQe coaga, IF-t pAint bu the Tyizy, f:7(k ant.onsw s! have cueg!Tor, 'a
wne﻿PGm fonkRDoqR7﻿wsp GG(" tet c2lithelidoyorvp,tMGL3om
aS.
g t d
Iw wand w HW. uneroofzwhe st ine ld bag"Afl?;To ewro
